In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os
import rdkit
from rdkit.Chem import AllChem, Draw, rdMolDescriptors
import io
from pathlib import Path

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml(
    "/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml"
)
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(
    CONFIG_ELN["url"], CONFIG_ELN["token"]
)

molecule_widgets = widgets.OpenbisObjectWidget("MoleculeConcept", DATA_MODEL)

molecule_structure = ipw.Image(
    value=utils.read_file(CONFIG["default_image_filepath"]),
    format="jpg",
    width="420px",
    height="450px",
    layout=ipw.Layout(border="solid 1px #cccccc"),
)

molecule_cdxml_file = ipw.FileUpload(multiple=False, accept=".cdxml")

molecule_support_files = ipw.FileUpload(multiple=True)

increase_buttons_size = utils.HTML(data="".join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)
quit_button = utils.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data='window.location.replace("home.ipynb")'))


def load_molecule_structure(change):
    for filename in molecule_cdxml_file.value:
        file_info = molecule_cdxml_file.value[filename]
        utils.save_file(file_info["content"], "structures/structure.cdxml")

    cdxml_molecule = utils.read_file("structures/structure.cdxml")
    molecules = rdkit.Chem.MolsFromCDXML(cdxml_molecule)

    if len(molecules) == 1:
        mol = molecules[0]  # Get first molecule
        mol_chemical_formula = rdMolDescriptors.CalcMolFormula(mol)  # Sum Formula
        mol_smiles = rdkit.Chem.MolToSmiles(mol)  # Canonical Smiles
        chem_mol = rdkit.Chem.MolFromSmiles(mol_smiles)

        if chem_mol is not None:
            AllChem.Compute2DCoords(chem_mol)  # Add coords to the atoms in the molecule
            img = Draw.MolToImage(chem_mol)
            buffer = io.BytesIO()
            img.save(buffer, format="PNG")
            molecule_structure.value = buffer.getvalue()
            img.save("structures/structure.png")
        else:
            print("Cannot generate molecule image.")

        molecule_widgets.properties_widgets["sum_formula"][
            "value_widget"
        ].value = mol_chemical_formula
        molecule_widgets.properties_widgets["smiles"]["value_widget"].value = mol_smiles

    elif len(molecules) > 1:
        print(f"There are more than one molecule in the file: {filename}")
    else:
        print(f"There are no molecules in the file: {filename}")


def create_molecule_openbis(b):
    # Check if the molecule is already in openBIS
    smiles_molecules_openbis = [
        molecule.props["smiles"]
        for molecule in utils.get_openbis_objects(
            OPENBIS_SESSION, type="MOLECULE_CONCEPT"
        )
    ]

    object_properties = utils.get_object_widget_values(
        molecule_widgets.properties_widgets
    )
    if object_properties["smiles"] in smiles_molecules_openbis:
        display(
            utils.Javascript(data="alert('Molecule Concept is already in openBIS!')")
        )

        if Path("structures/structure.png").exists():
            os.remove("structures/structure.png")

        if Path("structures/structure.png").exists():
            os.remove("structures/structure.cdxml")
    else:
        molecule_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type="MOLECULE_CONCEPT",
            collection="/MATERIALS/MOLECULES/PRECURSOR_COLLECTION",
            props=object_properties,
        )

        # Send molecule image to openBIS
        if Path("structures/structure.png").exists():
            # Upload cdxml
            utils.create_openbis_dataset(
                OPENBIS_SESSION,
                type="ELN_PREVIEW",
                object=molecule_object,
                file="structures/structure.png",
            )
            os.remove("structures/structure.png")

        if Path("structures/structure.png").exists():
            # Upload cdxml
            utils.create_openbis_dataset(
                OPENBIS_SESSION,
                type="RAW_DATA",
                object=molecule_object,
                file="structures/structure.cdxml",
            )
            os.remove("structures/structure.cdxml")

        # Additional support files
        utils.upload_datasets(
            OPENBIS_SESSION, molecule_object, molecule_support_files, "RAW_DATA"
        )

        display(utils.Javascript(data="alert('Upload successful!')"))

# Create molecule concept

## Upload CDXML file

In [ ]:
display(molecule_structure)
display(molecule_cdxml_file)
molecule_cdxml_file.observe(load_molecule_structure, names="value")

## Properties

In [ ]:
display(molecule_widgets)

## Support files

In [ ]:
display(molecule_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_molecule_openbis)
quit_button.on_click(close_notebook)